Ready data from numpy arrays stored on disk

In [1]:
using NPZ
coupling_matrix = npzread("./coupling_matrix.npy")
epi_array = npzread("./epi_array.npy")
mobilitypopulation_array_scaled = npzread("./mobilitypopulation_array_scaled.npy")
# batch dimension for FastDense layer is the second dimension
epi_array = permutedims(epi_array,[3,2,1])
mobilitypopulation_array_scaled = permutedims(mobilitypopulation_array_scaled,[2,1,3]);

In [2]:
println(size(coupling_matrix))
println(size(epi_array))
println(size(mobilitypopulation_array_scaled))

(104, 104, 99)
(104, 2, 99)
(7, 104, 99)


In [3]:
using DiffEqSensitivity, OrdinaryDiffEq, Zygote
using DiffEqFlux, Flux, Optim, OrdinaryDiffEq, Plots

┌ Info: Precompiling DiffEqFlux [aae7a2af-3d4f-5e19-a356-7da93b79d9d0]
└ @ Base loading.jl:1278
┌ Warning: Error requiring CUDA from DiffEqNoiseProcess:
│ UndefVarError: CuArrays not defined
│ Stacktrace:
│  [1] top-level scope at /Users/ibulu/.julia/packages/DiffEqNoiseProcess/Gu3FC/src/init.jl:9
│  [2] eval at ./boot.jl:331 [inlined]
│  [3] eval at /Users/ibulu/.julia/packages/DiffEqNoiseProcess/Gu3FC/src/DiffEqNoiseProcess.jl:1 [inlined]
│  [4] (::DiffEqNoiseProcess.var"#6#15")() at /Users/ibulu/.julia/packages/Requires/qy6zC/src/require.jl:85
│  [5] err(::Any, ::Module, ::String) at /Users/ibulu/.julia/packages/Requires/qy6zC/src/require.jl:42
│  [6] (::DiffEqNoiseProcess.var"#5#14")() at /Users/ibulu/.julia/packages/Requires/qy6zC/src/require.jl:84
│  [7] withpath(::Any, ::String) at /Users/ibulu/.julia/packages/Requires/qy6zC/src/require.jl:32
│  [8] (::DiffEqNoiseProcess.var"#4#13")() at /Users/ibulu/.julia/packages/Requires/qy6zC/src/require.jl:83
│  [9] #invokelatest#1 at ./es

## define a simple neural network / match JAX version

In [4]:
nn = FastChain(FastDense(7, 7, swish),
                       FastDense(7, 7, swish),
                       FastDense(7, 7, swish),
                       FastDense(7, 1, softplus));

In [5]:
# destructure neural network paraemeter into a vector. Conveniently, this matches with what we do in the JAX code
p0_nnet = initial_params(nn)
n_weights = length(p0_nnet);

###  let's apply the neural net defined above to the mobility data

In [6]:
nn(mobilitypopulation_array_scaled[:,:,1], p0_nnet);

## let,s define the parameters for scaling the coupling matrix county wise

In [7]:
using Random, Distributions

In [8]:
Random.seed!(123)
d = Normal()
n_counties = size(coupling_matrix,1)
p0_scaler = rand(d, n_counties);

## Let's define the coupled ode model with neural network

In [9]:
using ComponentArrays
using Parameters: @unpack

In [10]:
function SEIRD_mobility_coupled_outer(mobility_, coupling_matrix_, nn_)
    
    function SEIRD_mobility_coupled_inner(du, 
                                          u, 
                                          p_, t)
        s = @view u[:,1] 
        e = @view u[:,2] 
        id1 = @view u[:,3]
        id2 = @view u[:,4]
        id3 = @view u[:,5]
        id4 = @view u[:,6]
        id5 = @view u[:,7]
        id6 = @view u[:,8] 
        id7 = @view u[:,9] 
        d = @view u[:,10]
        ir1 = @view u[:,11]
        ir2 = @view u[:,12]
        ir3 = @view u[:,13]
        ir4 = @view u[:,14]
        ir5 = @view u[:,15]
        r = @view u[:,16]
        
        ds = @view du[:,1] 
        de = @view du[:,2] 
        did1 = @view du[:,3]
        did2 = @view du[:,4]
        did3 = @view du[:,5]
        did4 = @view du[:,6]
        did5 = @view du[:,7]
        did6 = @view du[:,8] 
        did7 = @view du[:,9] 
        dd = @view du[:,10]
        dir1 = @view du[:,11]
        dir2 = @view du[:,12]
        dir3 = @view du[:,13]
        dir4 = @view du[:,14]
        dir5 = @view du[:,15]
        dr = @view du[:,16]
        
        κ, α, γ = softplus.(p_[1:3])
    # κ*α and γ*η are not independent. The probablibility of transition from e to Ir and Id has to add up to 1
        η = - log(-expm1(-κ*α))/(γ+1.0e-8) 
        n_c = size(coupling_matrix_,1)
        scaler_ = softplus.(p_[4:3+n_c])
        cm_ = scaler_ .* coupling_matrix_[:,:,Int32(round(t+1.0))]
        p_nnet = p_[4+n_c:end]
        β = nn_(mobility_[:,:,Int32(round(t+1.0))], p_nnet)[1,:]
        i = id1+id2+id3+ir1+ir2+ir3+ir4+ir5
        c1 = (reshape(i,1,:)*transpose(cm_))[1,:]
        c2 = cm_*i
        a = β .* s .* i + β .* s .* (c1+c2)
        @. ds = -a
        @. de = a - κ*α*e - γ*η*e
        @. did1 = κ*(α*e-id1)
        @. did2 = κ*(id1-id2)
        @. did3 = κ*(id2-id3)
        @. did4 = κ*(id3-id4)
        @. did5 = κ*(id4-id5)
        @. did6 = κ*(id5-id6)
        @. did7 = κ*(id6-id7)
        @. dd = κ*id7
        
        @. dir1 = γ*(η*e-ir1)
        @. dir2 = γ*(ir1-ir2)
        @. dir3 = γ*(ir2-ir3)
        @. dir4 = γ*(ir3-ir4)
        @. dir5 = γ*(ir4-ir5)
        @. dr = γ*ir5
    end
end

SEIRD_mobility_coupled_outer (generic function with 1 method)

In [11]:
SEIRD_mobility_coupled = SEIRD_mobility_coupled_outer(mobilitypopulation_array_scaled, 
                                                      coupling_matrix, nn);

### define ode parameters

In [12]:
κ0_ = 0.97
α0_ = 0.00185
β0_ = 0.5
tb_ = 15
β1_ = 0.4
γ0_ = 0.24
inv_softplus(x) = x+log(-expm1(-x))    
p0_ode = inv_softplus.([κ0_, α0_, γ0_]);

### define initial conditions

In [13]:

ifr = 0.007
n_counties = size(coupling_matrix,1) 

n = ones(n_counties)
ic0 = epi_array[:,1,1]
d0 = epi_array[:,2,1]
r0 = d0/ifr
s0 = n-ic0-r0-d0
e0 = ic0
id10=id20=id30=id40=id50=id60=id70=ic0*ifr/7.0
ir10=ir20=ir30=ir40=ir50=ic0*(1.0-ifr)/5.0

u0 = hcat(s0, 
                    e0,
                    id10,id20,id30,id40,id50, id60, id70, d0,
                    ir10,ir20,ir30,ir40,ir50, r0);


combine all the model parameters

In [14]:
p_init = [p0_ode; p0_scaler; p0_nnet];

define time span and points to save the data

In [15]:
t_tot = Float32(size(coupling_matrix,3))
tspan = (0.0f0, t_tot-1.0)
tsteps = 0.0f0:1.0:t_tot-1.0

0.0:1.0:98.0

solve the ode with the initial conditions 

In [21]:
prob_seird = ODEProblem(SEIRD_mobility_coupled, u0, tspan, p_init)
sol_univ = solve(prob_seird, DP5(),abstol = 1e-6, reltol = 1e-3;saveat=tsteps);

In [22]:
function loss(params_)
    sol_ = solve(prob_seird, Tsit5(),p=params_, abstol = 1e-8, reltol = 1e-4, 
        sensealg=BacksolveAdjoint(),
           saveat=tsteps)
    sol = Array(sol_)

    l1 = sum((diff(sol[:,1,:],dims=2)-epi_array[:,1,2:end]).^2)
    l2 = sum((diff(sol[:,10,:],dims=2)-epi_array[:,2,2:end]).^2)
    return l1+l2
end

loss (generic function with 1 method)

In [23]:
using BenchmarkTools
using Zygote

In [24]:
@benchmark loss(p_init)

BenchmarkTools.Trial: 
  memory estimate:  349.58 MiB
  allocs estimate:  107891
  --------------
  minimum time:     288.021 ms (11.89% GC)
  median time:      308.054 ms (16.30% GC)
  mean time:        315.976 ms (15.57% GC)
  maximum time:     355.228 ms (14.51% GC)
  --------------
  samples:          16
  evals/sample:     1

In [25]:
@benchmark dp1 = Zygote.gradient(loss,p_init)

BenchmarkTools.Trial: 
  memory estimate:  14.01 GiB
  allocs estimate:  269620556
  --------------
  minimum time:     17.982 s (14.09% GC)
  median time:      17.982 s (14.09% GC)
  mean time:        17.982 s (14.09% GC)
  maximum time:     17.982 s (14.09% GC)
  --------------
  samples:          1
  evals/sample:     1